<a href="https://colab.research.google.com/github/maxmatical/fast.ai/blob/master/fastai_hf_self_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install fastai==2.2.7 # as of may 1 2020 fastai 2.3.0 breaks blurr
!pip install transformers
!pip install ohmeow-blurr==0.0.22

In [2]:
from fastai.test_utils import show_install
show_install()



```text
=== Software === 
python        : 3.7.10
fastai        : 2.2.7
fastcore      : 1.3.19
fastprogress  : 0.2.7
torch         : 1.7.1
nvidia driver : 460.32
torch cuda    : 10.2 / is available
torch cudnn   : 7605 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : Tesla T4

=== Environment === 
platform      : Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
distro        : #1 SMP Thu Jul 23 08:00:38 PDT 2020
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : 
/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Once in

In [3]:
from transformers import *
from fastai.text.all import *
from blurr.data.all import *
from blurr.modeling.all import *
import torch.nn as nn



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

imdb_df.shape

(1000, 3)

In [5]:
imdb_df.head(5)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


In [6]:
pretrained_model_name = 'distilroberta-base'
hf_model_cls = RobertaForSequenceClassification
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,
                                                                               model_cls=hf_model_cls)

In [7]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

dls = dblock.dataloaders(imdb_df, bs=4)

In [8]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,",negative
1,"Now that Che(2008) has finished its relatively short Australian cinema run (extremely limited release:1 screen in Sydney, after 6wks), I can guiltlessly join both hosts of ""At The Movies"" in taking Steven Soderbergh to task.<br /><br />It's usually satisfying to watch a film director change his style/subject, but Soderbergh's most recent stinker, The Girlfriend Experience(2009), was also missing a story, so narrative (and editing?) seem to suddenly be Soderbergh's main challenge. Strange, after 20-odd years in the business. He was probably never much good at narrative, just hid it well inside ""edgy"" projects.<br /><br />None of this excuses him this present, almost diabolical failure. As David Stratton warns, ""two parts of Che don't (even) make a whole"". <br /><br />Epic biopic in name only, Che(2008) barely qualifies as a feature film! It certainly has no legs, inasmuch as except for its uncharacteristic ultimate resolution forced upon it by history, Soderbergh's 4.5hrs-long dirge just goes nowhere.<br /><br />Even Margaret Pomeranz, the more forgiving of Australia's At The Movies duo, noted about Soderbergh's repetitious waste of (HD digital storage): ""you're in the woods...you're in the woods...you're in the woods..."". I too am surprised Soderbergh didn't give us another 2.5hrs of THAT somewhere between his existing two Parts, because he still left out massive chunks of Che's ""revolutionary"" life! <br /><br />For a biopic of an important but infamous historical figure, Soderbergh unaccountably alienates, if not deliberately insults, his audiences by<br /><br />1. never providing most of Che's story; <br /><br />2. imposing unreasonable film lengths with mere dullard repetition; <br /><br />3. 

### Training 1st model

In [9]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=LabelSmoothingCrossEntropyFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="accuracy", fname="model_1")
]



In [10]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)



epoch,train_loss,valid_loss,accuracy,time
0,0.688996,0.672182,0.780000,00:15
1,0.477257,0.407832,0.875000,00:14
2,0.347807,0.374730,0.915000,00:15


Better model found at epoch 0 with accuracy value: 0.7799999713897705.
Better model found at epoch 1 with accuracy value: 0.875.
Better model found at epoch 2 with accuracy value: 0.9150000214576721.


### generate psuedo-labels for both train

use 1 hot encoded for validation data

In [12]:
df_train = imdb_df[imdb_df["is_valid"]==False]

In [13]:
test_dl = learn.dls.test_dl(list(df_train["text"]))

In [14]:
pred_proba, _ = learn.get_preds(dl=test_dl, with_decoded=False)

In [15]:
pred_proba = pred_proba.cpu().numpy()

In [16]:
pred_proba.shape

(800, 2)

In [17]:
pred_proba[1]

array([0.17050801, 0.82949203], dtype=float32)

`learn.get_preds` defaults to activation associated with the loss function, so applying softmax to the logits

In [18]:
# get the mapping from class labels to index of predicted probabilities
learn.dls.vocab

['negative', 'positive']

In [11]:
# old code, leave commented for now because it might be useful in the future

# pred_proba_df = pd.DataFrame(pred_proba, columns=["negative", "positive"])
# df_pseudolabels = pd.concat([imdb_df, pred_proba_df], axis=1)
# df_pseudolabels.head(5)

add pseudo labels for training data

In [32]:
pred_proba_df_train = pd.DataFrame(pred_proba, columns=["negative", "positive"])
df_pseudolabels_train = pd.concat([df_train, pred_proba_df_train], axis=1)
df_pseudolabels_train.head(5)

,label,text,is_valid,negative,positive
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False,0.902180,0.097820
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False,0.170508,0.829492
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False,0.933249,0.066751
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False,0.506622,0.493378
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False,0.842215,0.157785


one-hot encode val df

In [30]:
df_val = imdb_df[imdb_df["is_valid"]==True]
pred_proba_df_val = pd.get_dummies(df_val["label"])
df_pseudolabels_val = pd.concat([df_val, pred_proba_df_val], axis=1)
df_pseudolabels_val.head(5)

,label,text,is_valid,negative,positive
800,positive,"This very funny British comedy shows what might happen if a section of London, in this case Pimlico, were to declare itself independent from the rest of the UK and its laws, taxes & post-war restrictions. Merry mayhem is what would happen.<br /><br />The explosion of a wartime bomb leads to the discovery of ancient documents which show that Pimlico was ceded to the Duchy of Burgundy centuries ago, a small historical footnote long since forgotten. To the new Burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from Whitehall.<br /><br />...",True,0,1
801,positive,"I saw this movie once as a kid on the late-late show and fell in love with it.<br /><br />It took 30+ years, but I recently did find it on DVD - it wasn't cheap, either - in a catalog that specialized in war movies. We watched it last night for the first time. The audio was good, however it was grainy and had the trailers between reels. Even so, it was better than I remembered it. I was also impressed at how true it was to the play.<br /><br />The catalog is around here someplace. If you're sincere in finding it, fire me a missive and I'll see if I can get you the info. cartwrightbride@yah...",True,0,1
802,positive,"This is, in my opinion, a very good film, especially for Michael Jackson lovers. It contains a message on drugs, stunning special effects, and an awesome music video.<br /><br />The main film is centered around the song and music video 'Smooth Criminal.' Unlike the four-minute music video, it is normal speed and, in my opinion, much easier to watch.<br /><br />The plot is rather weird, however. Michael Jackson plays a magical 'gangster' that, when he sees a shooting star, he transforms into a piece of machinery. Throughout the film, he transforms into a race car, a giant robot, and a space...",True,0,1
803,positive,"In Iran, women are not permitted to attend men's sporting events, apparently to ""protect"" them from all the cursing and foul language they might hear emanating from the male fans (so since men can't restrain or behave themselves, women are forced to suffer. Go figure.). ""Offside"" tells the tale of a half dozen or so young women who, dressed like men, attempt to sneak into the high-stakes match between Iran and Bahrain that, in 2005, qualified Iran to go to the World Cup (the movie was actually filmed in large part during that game).<br /><br />""Offside"" is a slice-of-life comedy that will ...",True,0,1
804,positive,"""In April 1946, the University of Chicago agreed to operate Argonne National Laboratory, with an association of Midwestern universities offering to sponsor the research. Argonne thereby became the first ""national"" laboratory. It did not, however, remain at its original location in the Argonne forest. In 1947, it moved farther west from the ""Windy City"" to a new site on Illinois farmland. When Alvin Weinberg visited Argonne's director, Walter Zinn, in 1947, he asked him what kind of reactor was to be built at the new site. When Zinn described a heavy-water reactor operating at one-tenth the...",True,0,1


In [33]:
# combine the 2 dfs
df_pseudolabels = pd.concat([df_pseudolabels_train, df_pseudolabels_val], axis=0, ignore_index=True)
df_pseudolabels.head(5)

,label,text,is_valid,negative,positive
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False,0.902180,0.097820
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False,0.170508,0.829492
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False,0.933249,0.066751
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False,0.506622,0.493378
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False,0.842215,0.157785


In [34]:
df_pseudolabels.tail(5)

,label,text,is_valid,negative,positive
995,negative,"There are many different versions of this one floating around, so make sure you can locate one of the unrated copies, otherwise some gore and one scene of nudity might be missing. Some versions also omit most of the opening sequence and other bits here and there. The cut I saw has the on-screen title WITCHCRAFT: EVIL ENCOUNTERS and was released by Shriek Show, who maintain the original US release title WITCHERY for the DVD release. It's a nice-looking print and seems to have all of the footage, but has some cropping/aspect ratio issues. In Italy, it was released as LA CASA 4 (WITCHCRAFT). ...",True,1.0,0.0
996,positive,"Once upon a time Hollywood produced live-action, G-rated movies without foul language, immorality, and gore-splattered violence. These movies neither insulted your intelligence no manipulated your emotions. The heroes differed little from the crowd. They shared the same feelings and bore the same burdens. Since the 1970s, the film industry has pretty much written off G-rated movies for adults. Basically, modern mature audiences demand large doses of embellished realism for their cinematic diet, laced heavily with vile profanity, mattress-thumping sex, and knuckle-bruising fisticuffs. These...",True,0.0,1.0
997,negative,"Wenders was great with Million $ Hotel.I don't know how he came up with this film! The idea of giving the situation after spt11 and the view of American Society is hopeful,that makes it 2 out of ten.But this is not a movie.Is that the best someone can do with a great idea(the west-east clash).There are important things going on in middle east and it is just issued on the screen of a MAC* with the fingers of an Amerian girl who is actually at the level of stupidity(because she is just ignorant about the facts).The characters are not well shaped.And the most important thing is the idea that ...",True,1.0,0.0
998,negative,"Although a film with Bruce Willis is always worth watching, you better skip this one. I watched this one on television, so I didn't have to plunk down cash for it. Lucky me.<br /><br />The plot develops slowly, very slowly. Although the first 30 minutes or so are quite believable, it gets more and more unbelievable towards the end. It is highly questionable, if a seasoned soldier like Lt. Waters would disobey direct orders. And even if he would, if the rest of his platoon would. They know he puts them in direct danger, and they know they will certainly die if they follow him, but what the ...",True,1.0,0.0
999,positive,"A compelling, honest, daring, and unforgettable psychological horror film that touches on the painful experiences of pain caused by rape - ""Descent"" is a film that went under-the-radar due to its lack of distribution because, frankly, the film is so brutal in its depictions, that if it had been released theatrically, it may have met itself to some strong biased hate.<br /><br />The film deserves to be discovered for, not only its dark themes, and not only for its amazing direction and authentic style - but most of all for its performances. Chad Faust is absolutely stunning, bringing enough...",True,0.0,1.0


## Train on pseudolabels

In [35]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), RegressionBlock) # use regression block for regression tasks
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader(['negative', 'positive']), splitter=ColSplitter()) # multiple columns for probabilities

dls = dblock.dataloaders(df_pseudolabels, bs=4)

In [36]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,text_
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,","(0.43151307106018066, 0.5684869289398193)"
1,"This film sat on my Tivo for weeks before I watched it. I dreaded a self-indulgent yuppie flick about relationships gone bad. I was wrong; this was an engrossing excursion into the screwed-up libidos of New Yorkers.<br /><br />The format is the same as Max Ophuls' ""La Ronde,"" based on a play by Arthur Schnitzler, who is given an ""inspired by"" credit. It starts from one person, a prostitute, standing on a street corner in Brooklyn. She is picked up by a home contractor, who has sex with her on the hood of a car, but can't come. He refuses to pay her. When he's off peeing, she answers his cell phone and takes a message. She runs away with his keys.<br /><br />Then the story switches to the contractor, who pays a professional call on a rich, bored New York woman, who plays with him until he is aroused, then she pulls away. She tells him how desperate and unhappy she is; he tells her how beautiful she is, and lucky. As he is leaving, she asks if he would have sex with her. She sits on top of him, bounces up and down. This time he comes, the he leaves.<br /><br />The woman and her husband throw a dinner party for their trendy friends. Hubby (Robert) is talking business, wife (Ellen) is bored, and switches the subject to sex, and how often men and women think about it. Husband switches conversation to desert. Later, after the guests leave, Ellen tries to entice Robert into sex. Robert wants none of it, and puts on a jazz record. Ellen turns on the radio; Robert turns up the music; Ellen turns on the TV; Robert turns on another TV. Cacophony ensues. Ellen goes up on the roof, Robert joins her. Ellen confesses that she needs to experience more men, men other than Robert. Robert 

In [37]:
# accuracy for pseudolabels
def pseudolabel_accuracy(pred, target):
    return (pred.argmax(dim=1) == target.argmax(dim=1)).float().mean()


In [38]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=MSELossFlat(),
                metrics=[pseudolabel_accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

# maybe this should be (training?) loss?
cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='pseudolabel_accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='pseudolabel_accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="pseudolabel_accuracy", fname="model_1")
]



In [39]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)


epoch,train_loss,valid_loss,pseudolabel_accuracy,time
0,0.084418,0.110880,0.895000,00:15
1,0.021435,0.086709,0.895000,00:15
2,0.017692,0.085655,0.900000,00:15


Better model found at epoch 0 with pseudolabel_accuracy value: 0.8949999809265137.
Better model found at epoch 2 with pseudolabel_accuracy value: 0.8999999761581421.


check results

In [40]:
df_pseudolabels.head(5)

,label,text,is_valid,negative,positive
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False,0.902180,0.097820
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False,0.170508,0.829492
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False,0.933249,0.066751
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False,0.506622,0.493378
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False,0.842215,0.157785


In [41]:
test_dl = learn.dls.test_dl(list(imdb_df["text"]))
pred_proba, _ = learn.get_preds(dl=test_dl, with_decoded=False)

In [42]:
pred_proba = pred_proba.cpu().numpy()
pred_proba.shape

(1000, 2)

test accuracy with actual labels

In [43]:
preds = pred_proba.argmax(axis=1)
pred_labels = ["negative" if p == 0 else "positive" for p in preds ]
print(len(pred_labels))
pred_labels[:5]


1000


['negative', 'positive', 'negative', 'positive', 'negative']

In [44]:
df_distilled_preds = pd.DataFrame(data=pred_labels, columns=["self_distilled_pred_label"])
df_performance = pd.concat([imdb_df, df_distilled_preds], axis=1)
df_performance_val = df_performance[df_performance["is_valid"]==True]
df_performance_val.head(5)

,label,text,is_valid,self_distilled_pred_label
800,positive,"This very funny British comedy shows what might happen if a section of London, in this case Pimlico, were to declare itself independent from the rest of the UK and its laws, taxes & post-war restrictions. Merry mayhem is what would happen.<br /><br />The explosion of a wartime bomb leads to the discovery of ancient documents which show that Pimlico was ceded to the Duchy of Burgundy centuries ago, a small historical footnote long since forgotten. To the new Burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from Whitehall.<br /><br />...",True,positive
801,positive,"I saw this movie once as a kid on the late-late show and fell in love with it.<br /><br />It took 30+ years, but I recently did find it on DVD - it wasn't cheap, either - in a catalog that specialized in war movies. We watched it last night for the first time. The audio was good, however it was grainy and had the trailers between reels. Even so, it was better than I remembered it. I was also impressed at how true it was to the play.<br /><br />The catalog is around here someplace. If you're sincere in finding it, fire me a missive and I'll see if I can get you the info. cartwrightbride@yah...",True,positive
802,positive,"This is, in my opinion, a very good film, especially for Michael Jackson lovers. It contains a message on drugs, stunning special effects, and an awesome music video.<br /><br />The main film is centered around the song and music video 'Smooth Criminal.' Unlike the four-minute music video, it is normal speed and, in my opinion, much easier to watch.<br /><br />The plot is rather weird, however. Michael Jackson plays a magical 'gangster' that, when he sees a shooting star, he transforms into a piece of machinery. Throughout the film, he transforms into a race car, a giant robot, and a space...",True,positive
803,positive,"In Iran, women are not permitted to attend men's sporting events, apparently to ""protect"" them from all the cursing and foul language they might hear emanating from the male fans (so since men can't restrain or behave themselves, women are forced to suffer. Go figure.). ""Offside"" tells the tale of a half dozen or so young women who, dressed like men, attempt to sneak into the high-stakes match between Iran and Bahrain that, in 2005, qualified Iran to go to the World Cup (the movie was actually filmed in large part during that game).<br /><br />""Offside"" is a slice-of-life comedy that will ...",True,positive
804,positive,"""In April 1946, the University of Chicago agreed to operate Argonne National Laboratory, with an association of Midwestern universities offering to sponsor the research. Argonne thereby became the first ""national"" laboratory. It did not, however, remain at its original location in the Argonne forest. In 1947, it moved farther west from the ""Windy City"" to a new site on Illinois farmland. When Alvin Weinberg visited Argonne's director, Walter Zinn, in 1947, he asked him what kind of reactor was to be built at the new site. When Zinn described a heavy-water reactor operating at one-tenth the...",True,negative


In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [46]:
accuracy_score(df_performance_val["label"], df_performance_val["self_distilled_pred_label"])

0.9

In [47]:
confusion_matrix(df_performance_val["label"], df_performance_val["self_distilled_pred_label"])

array([[101,   6],
       [ 14,  79]])

# Notes

1. should psuedolabels be generated for only training data?
Thought:
- only get pseudo-label for train data
- use 1 hot encoding for validation data
- optimize for train MSE/MAE (looking at validation MSE isn't important since it's one hot encoded)
- callback monitors, `train_loss` or `pseudolabel_accuracy`?

2. if pseudolabels should be generated for both train and val data
- optimize for MSE then
- this seems like the wrong way to go about it?
